## Quantum Broadcasting of An Arbitary n-Qubit Cluster State: Quantum Noise Analysis Using Quantum State Tomography Via IBMQ Simulation


Yousef Mafi$^{1}$. Ali Kookani$^{2}$. 

$^{1}$ Department of Electrical and Computer Engineering, University of Tehran, Tehran, Iran.

$^{2}$ Department of Engineering, College of Farabi, University of Tehran, Tehran, Iran

**Abstract**

This study focuses on the challenges posed by quantum noise to communication protocols, with a particular emphasis on the vulnerabilities of current quantum broadcast protocols. The research employs cluster states, known for their multipartite entanglement, as diagnostic tools to understand and address the impact of quantum noise on transmitted information. Implementing two novel controlled quantum broadcasting protocols on the IBM Quantum Experience platform, using Qiskit and the QASM simulator, the investigation evaluates the effects of Amplitude-damping (AD) and Phase-damping (PD) noises through Kraus operator models. A notable aspect of the study is the pioneering use of quantum state tomography for a comprehensive analysis of quantum noise effects, providing novel insights into the robustness of quantum communication protocols.

**Keywords**: Quantum Broadcast Protocol. IBMQ. Qiskit. Quantum Noise. Quantum State Tomography.



In [ ]:
# Import necessary libraries and modules
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Kraus, SuperOp
from qiskit.providers.aer import AerSimulator, QasmSimulator
from qiskit.tools.visualization import plot_histogram
from qiskit.visualization import plot_bloch_multivector
from numpy import pi, random
from math import sqrt, atan2, cos, sin, pow
from qiskit import *
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from qiskit.extensions import *
from qiskit.circuit.library.standard_gates import RYGate
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import QuantumError
import qiskit_aer.noise as noise
from qiskit.providers.aer.noise import amplitude_damping_error, phase_damping_error, phase_amplitude_damping_error

In [2]:
def Quantum_Protocol(theta):
    # Define desired states
    a = sqrt(1/2)*(cos(theta/2)+sin(theta/2))
    b = sqrt(1/2)*(cos(theta/2)-sin(theta/2))
    # Define U_M operator
    rows, cols = (16, 16)
    arr = [[0]*cols]*rows
    
    U_M = np.array(arr)
    for x in range(16): 
        U_M[x,x] = 1
    
    U_M=np.float64(U_M)
    
    U_M[0,0]  = a**2
    U_M[3,0]  = a*b
    U_M[12,0] = a*b
    U_M[15,0] = b**2
    
    U_M[0,3]  = a*b
    U_M[3,3]  = -a**2
    U_M[12,3] = b**2
    U_M[15,3] = -a*b
    
    U_M[0,12]  = a*b
    U_M[3,12]  = b**2
    U_M[12,12] = -a**2
    U_M[15,12] = -a*b
    
    U_M[0,15]  = b**2
    U_M[3,15]  = -a*b
    U_M[12,15] = -a*b
    U_M[15,15] = a**2
    
    U_M_gate = UnitaryGate(U_M,label='Um')
    
    # Define Classica and Quantum bits
    q = QuantumRegister(8, 'q')
    c = ClassicalRegister(8,'c')
    Qc = QuantumCircuit(q,c)
    
    # Prepare GHZ states
    Qc.h(q[0])
    Qc.cx(q[0],q[1])
    Qc.h(q[2])
    Qc.cx(q[2],q[3])
    Qc.cx(q[0],q[4])
    Qc.cx(q[1],q[5])
    Qc.cx(q[2],q[6])
    Qc.cx(q[3],q[7])
    Qc.barrier(q)
    
    # Change basis measurement to the orthonormal basis {|ξ(0)⟩, |ξ(1)⟩, |ξ(2)⟩, |ξ(3)⟩}
    Qc.append(U_M_gate, [q[0], q[1],q[2], q[3]])
    
    # Measure Alice's qubits
    Qc.measure(q[0],c[0])
    Qc.measure(q[1],c[1])
    Qc.measure(q[2],c[2])
    Qc.measure(q[3],c[3])
    Qc.barrier(q)
    
    # Apply U gate on Bob and Chalie's qubits qubits
    # Measurement Results: |ξ(0)⟩
    # No-action
    
    # Measurement Results: |ξ(1)⟩
    Qc.z(q[4]).c_if(c,3)
    Qc.x(q[4]).c_if(c,3)
    Qc.x(q[5]).c_if(c,3)
    
    # Measurement Results: |ξ(2)⟩
    Qc.z(q[6]).c_if(c,12)
    Qc.x(q[6]).c_if(c,12)
    Qc.x(q[7]).c_if(c,12)
    
    # Measurement Results: |ξ(3)⟩
    Qc.z(q[4]).c_if(c,15)
    Qc.z(q[6]).c_if(c,15)
    Qc.x(q[4]).c_if(c,15)
    Qc.x(q[5]).c_if(c,15)
    Qc.x(q[6]).c_if(c,15)
    Qc.x(q[7]).c_if(c,15)
    
    Qc.barrier(q)

    return Qc,q,c,a,b

In [3]:
def AD_Noise(eta_AD):
    # Amplitude-damping noise model
    # noise_model_AD an empty noise model
    noise_model_AD = NoiseModel()
    
    # Add Amplitude-damping error to channel qubits
    error_qc = amplitude_damping_error(eta_AD, 0)
    error_qt = amplitude_damping_error(eta_AD, 0)
    CNOT_error = error_qt.tensor(error_qc)
    noise_model_AD.add_quantum_error(CNOT_error, 'cx', [0, 4])
    noise_model_AD.add_quantum_error(CNOT_error, 'cx', [1, 5])
    noise_model_AD.add_quantum_error(CNOT_error, 'cx', [2, 6])
    noise_model_AD.add_quantum_error(CNOT_error, 'cx', [3, 7])
    
    # Print noise model info
    print(noise_model_AD)
    return noise_model_AD

In [4]:
def PD_Noise(eta_PD):
    # Phase-damping noise model
    # Create an empty noise model
    noise_model_PD = NoiseModel()
    
    # Add Phase-damping error to channel qubits
    error_qc = phase_damping_error(eta_PD, 0)
    error_qt = phase_damping_error(eta_PD, 0)
    CNOT_error = error_qt.tensor(error_qc)
    noise_model_PD.add_quantum_error(CNOT_error, 'cx', [0, 4])
    noise_model_PD.add_quantum_error(CNOT_error, 'cx', [1, 5])
    noise_model_PD.add_quantum_error(CNOT_error, 'cx', [2, 6])
    noise_model_PD.add_quantum_error(CNOT_error, 'cx', [3, 7])
    # Print noise model info
    print(noise_model_PD)
    return noise_model_PD

In [48]:
def PAD_Noise(eta_PAD):
    # Phase Amplitude-damping noise model
    # Create an empty noise model
    noise_model_PAD = NoiseModel()
    
    # Add Phase Amplitude-damping error to channel qubits
    error_qc = phase_amplitude_damping_error(eta_PAD, eta_PAD)
    error_qt = phase_amplitude_damping_error(eta_PAD, eta_PAD)
    CNOT_error = error_qt.tensor(error_qc)
    noise_model_PAD.add_quantum_error(CNOT_error, 'cx', [0, 4])
    noise_model_PAD.add_quantum_error(CNOT_error, 'cx', [1, 5])
    noise_model_PAD.add_quantum_error(CNOT_error, 'cx', [2, 6])
    noise_model_PAD.add_quantum_error(CNOT_error, 'cx', [3, 7])
    # Print noise model info
    print(noise_model_PAD)
    return noise_model_PAD

In [6]:
def Simulation_Noisy(Qc,NoiseModel):
    # Set the number of shots for the simulation
    m = 1000
    
    # Choose the QasmSimulator for the simulation with noise model
    simulator = QasmSimulator(noise_model=NoiseModel)
    
    # Transpile the quantum circuit for execution on the simulator
    circ = transpile(Qc, simulator)
    
    # Run the simulation with the specified number of shots
    result = simulator.run(circ, shots=m).result()
    
    # Obtain the measurement outcomes and their counts
    counts = result.get_counts(circ)
    
    return counts

In [7]:
def Output_state(counts):
    # Initialize an array to store counts for different states
    S_C = np.zeros([4])
    
    # Count occurrences of different states based on measurement outcomes
    for state in counts:
        if (state[0] == '0') and (state[1] == '0'):
            S_C[0] = S_C[0] + counts[state]
        if (state[0] == '0') and (state[1] == '1'):
            S_C[1] = S_C[1] + counts[state]
        if (state[0] == '1') and (state[1] == '0'):
            S_C[2] = S_C[2] + counts[state]
        if (state[0] == '1') and (state[1] == '1'):
            S_C[3] = S_C[3] + counts[state]
    
    # Create a dictionary to store counts for specific states
    count_Charlie = {'00': S_C[0], '01': S_C[1], '10': S_C[2], '11': S_C[3]}
    return count_Charlie

In [68]:
noise_model = PAD_Noise(eta_PAD=0.4)
theta=pi/2

# Quantum State Tomography
for i in ['I','X','Y','Z']:
    for j in ['I','X','Y','Z']:
        Qc,q,c,a,b = Quantum_Protocol(theta)
        if((i=='I')&(j=='I')):
            # Apply Quantum State Tomography Gates I,I
            Qc.id(q[6])
            Qc.id(q[7])
            Qc.measure(q[4],c[4])
            Qc.measure(q[5],c[5])
            Qc.measure(q[6],c[6])
            Qc.measure(q[7],c[7])
            counts=Simulation_Noisy(Qc,noise_model)
            count_Charlie = Output_state(counts)
            tr_II = count_Charlie['00'] / 1000+count_Charlie['11'] / 1000+count_Charlie['01'] / 1000+count_Charlie['10'] / 1000
        if((i=='I')&(j=='X')):
            # Apply Quantum State Tomography Gates I,X
            Qc.id(q[6])
            Qc.h(q[7])
            Qc.measure(q[4],c[4])
            Qc.measure(q[5],c[5])
            Qc.measure(q[6],c[6])
            Qc.measure(q[7],c[7])
            counts=Simulation_Noisy(Qc,noise_model)
            count_Charlie = Output_state(counts)
            tr_IX = count_Charlie['00'] / 1000-count_Charlie['11'] / 1000+count_Charlie['01'] / 1000-count_Charlie['10'] / 1000
        if((i=='I')&(j=='Y')):
            # Apply Quantum State Tomography Gates I,Y
            Qc.id(q[6])
            Qc.sdg(q[7])
            Qc.h(q[7])
            Qc.measure(q[4],c[4])
            Qc.measure(q[5],c[5])
            Qc.measure(q[6],c[6])
            Qc.measure(q[7],c[7])
            counts=Simulation_Noisy(Qc,noise_model)
            count_Charlie = Output_state(counts)
            tr_IY = count_Charlie['00'] / 1000-count_Charlie['11'] / 1000+count_Charlie['01'] / 1000-count_Charlie['10'] / 1000
        if((i=='I')&(j=='Z')):
            # Apply Quantum State Tomography Gates I,Z
            Qc.id(q[6])
            Qc.id(q[7])
            Qc.measure(q[4],c[4])
            Qc.measure(q[5],c[5])
            Qc.measure(q[6],c[6])
            Qc.measure(q[7],c[7])
            counts=Simulation_Noisy(Qc,noise_model)
            count_Charlie = Output_state(counts)
            tr_IZ = count_Charlie['00'] / 1000-count_Charlie['11'] / 1000+count_Charlie['01'] / 1000-count_Charlie['10'] / 1000
        if((i=='X')&(j=='I')):
            # Apply Quantum State Tomography Gates X,I
            Qc.h(q[6])
            Qc.id(q[7])
            Qc.measure(q[4],c[4])
            Qc.measure(q[5],c[5])
            Qc.measure(q[6],c[6])
            Qc.measure(q[7],c[7])
            counts=Simulation_Noisy(Qc,noise_model)
            count_Charlie = Output_state(counts)
            tr_XI = count_Charlie['00'] / 1000-count_Charlie['11'] / 1000-count_Charlie['01'] / 1000+count_Charlie['10'] / 1000
        if((i=='X')&(j=='X')):
            # Apply Quantum State Tomography Gates X,X
            Qc.h(q[6])
            Qc.h(q[7])
            Qc.measure(q[4],c[4])
            Qc.measure(q[5],c[5])
            Qc.measure(q[6],c[6])
            Qc.measure(q[7],c[7])
            counts=Simulation_Noisy(Qc,noise_model)
            count_Charlie = Output_state(counts)
            tr_XX = count_Charlie['00'] / 1000+count_Charlie['11'] / 1000-count_Charlie['01'] / 1000-count_Charlie['10'] / 1000
        if((i=='X')&(j=='Y')):
            # Apply Quantum State Tomography Gates X,Y
            Qc.h(q[6])
            Qc.sdg(q[7])
            Qc.h(q[7])
            Qc.measure(q[4],c[4])
            Qc.measure(q[5],c[5])
            Qc.measure(q[6],c[6])
            Qc.measure(q[7],c[7])
            counts=Simulation_Noisy(Qc,noise_model)
            count_Charlie = Output_state(counts)
            tr_XY = count_Charlie['00'] / 1000+count_Charlie['11'] / 1000-count_Charlie['01'] / 1000-count_Charlie['10'] / 1000
        if((i=='X')&(j=='Z')):
            # Apply Quantum State Tomography Gates X,Z
            Qc.h(q[6])
            Qc.id(q[7])
            Qc.measure(q[4],c[4])
            Qc.measure(q[5],c[5])
            Qc.measure(q[6],c[6])
            Qc.measure(q[7],c[7])
            counts=Simulation_Noisy(Qc,noise_model)
            count_Charlie = Output_state(counts)
            tr_XZ = count_Charlie['00'] / 1000+count_Charlie['11'] / 1000-count_Charlie['01'] / 1000-count_Charlie['10'] / 1000
        if((i=='Y')&(j=='I')):
            # Apply Quantum State Tomography Gates Y,I
            Qc.sdg(q[6])
            Qc.h(q[6])
            Qc.id(q[7])
            Qc.measure(q[4],c[4])
            Qc.measure(q[5],c[5])
            Qc.measure(q[6],c[6])
            Qc.measure(q[7],c[7])
            counts=Simulation_Noisy(Qc,noise_model)
            count_Charlie = Output_state(counts)
            tr_YI = count_Charlie['00'] / 1000-count_Charlie['11'] / 1000-count_Charlie['01'] / 1000+count_Charlie['10'] / 1000
        if((i=='Y')&(j=='X')):
            # Apply Quantum State Tomography Gates Y,X
            Qc.sdg(q[6])
            Qc.h(q[6])
            Qc.h(q[7])
            Qc.measure(q[4],c[4])
            Qc.measure(q[5],c[5])
            Qc.measure(q[6],c[6])
            Qc.measure(q[7],c[7])
            counts=Simulation_Noisy(Qc,noise_model)
            count_Charlie = Output_state(counts)
            tr_YX = count_Charlie['00'] / 1000+count_Charlie['11'] / 1000-count_Charlie['01'] / 1000-count_Charlie['10'] / 1000
        if((i=='Y')&(j=='Y')):
            # Apply Quantum State Tomography Gates Y,Y
            Qc.sdg(q[6])
            Qc.h(q[6])
            Qc.sdg(q[7])
            Qc.h(q[7])
            Qc.measure(q[4],c[4])
            Qc.measure(q[5],c[5])
            Qc.measure(q[6],c[6])
            Qc.measure(q[7],c[7])
            counts=Simulation_Noisy(Qc,noise_model)
            count_Charlie = Output_state(counts)
            tr_YY = count_Charlie['00'] / 1000+count_Charlie['11'] / 1000-count_Charlie['01'] / 1000-count_Charlie['10'] / 1000
        if((i=='Y')&(j=='Z')):
            # Apply Quantum State Tomography Gates Y,Z
            Qc.sdg(q[6])
            Qc.h(q[6])
            Qc.id(q[7])
            Qc.measure(q[4],c[4])
            Qc.measure(q[5],c[5])
            Qc.measure(q[6],c[6])
            Qc.measure(q[7],c[7])
            counts=Simulation_Noisy(Qc,noise_model)
            count_Charlie = Output_state(counts)
            tr_YZ = count_Charlie['00'] / 1000+count_Charlie['11'] / 1000-count_Charlie['01'] / 1000-count_Charlie['10'] / 1000
        if((i=='Z')&(j=='I')):
            # Apply Quantum State Tomography Gates Z,I
            Qc.id(q[6])
            Qc.id(q[7])
            Qc.measure(q[4],c[4])
            Qc.measure(q[5],c[5])
            Qc.measure(q[6],c[6])
            Qc.measure(q[7],c[7])
            counts=Simulation_Noisy(Qc,noise_model)
            count_Charlie = Output_state(counts)
            tr_ZI = count_Charlie['00'] / 1000-count_Charlie['11'] / 1000-count_Charlie['01'] / 1000+count_Charlie['10'] / 1000
        if((i=='Z')&(j=='X')):
            # Apply Quantum State Tomography Gates Z,X
            Qc.id(q[6])
            Qc.h(q[7])
            Qc.measure(q[4],c[4])
            Qc.measure(q[5],c[5])
            Qc.measure(q[6],c[6])
            Qc.measure(q[7],c[7])
            counts=Simulation_Noisy(Qc,noise_model)
            count_Charlie = Output_state(counts)
            tr_ZX = count_Charlie['00'] / 1000+count_Charlie['11'] / 1000-count_Charlie['01'] / 1000-count_Charlie['10'] / 1000
        if((i=='Z')&(j=='Y')):
            # Apply Quantum State Tomography Gates Z,Y
            Qc.id(q[6])
            Qc.sdg(q[7])
            Qc.h(q[7])
            Qc.measure(q[4],c[4])
            Qc.measure(q[5],c[5])
            Qc.measure(q[6],c[6])
            Qc.measure(q[7],c[7])
            counts=Simulation_Noisy(Qc,noise_model)
            count_Charlie = Output_state(counts)
            tr_ZY = count_Charlie['00'] / 1000+count_Charlie['11'] / 1000-count_Charlie['01'] / 1000-count_Charlie['10'] / 1000
        if((i=='Z')&(j=='Z')):
            # Apply Quantum State Tomography Gates Z,Z
            Qc.id(q[6])
            Qc.id(q[7])
            Qc.measure(q[4],c[4])
            Qc.measure(q[5],c[5])
            Qc.measure(q[6],c[6])
            Qc.measure(q[7],c[7])
            counts=Simulation_Noisy(Qc,noise_model)
            count_Charlie = Output_state(counts)
            tr_ZZ = count_Charlie['00'] / 1000+count_Charlie['11'] / 1000-count_Charlie['01'] / 1000-count_Charlie['10'] / 1000
        
            
            
            
I = np.array([[1,0],[0,1]])
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])
II = np.kron(I,I)
IX = np.kron(I,X)
IY = np.kron(I,Y)
IZ = np.kron(I,Z)
XI = np.kron(X,I)
XX = np.kron(X,X)
XY = np.kron(X,Y)
XZ = np.kron(X,Z)
YI = np.kron(Y,I)
YX = np.kron(Y,X)
YY = np.kron(Y,Y)
YZ = np.kron(Y,Z)
ZI = np.kron(Z,I)
ZX = np.kron(Z,X)
ZY = np.kron(Z,Y)
ZZ = np.kron(Z,Z)

# Desity Matrix Estimation
density_matrix = 1/4*(tr_II*II+tr_IX*IX+tr_IY*IY+tr_IZ*IZ+tr_XI*XI+tr_XX*XX+tr_XY*XY+tr_XZ*XZ+tr_YI*YI+tr_YX*YX+tr_YY*YY+tr_YZ*YZ
                      +tr_ZI*ZI+tr_ZX*ZX+tr_ZY*ZY+tr_ZZ*ZZ)
print("Desity Matrix:",density_matrix)

# The desired state
a = sqrt(1/2)*(cos(theta/2)+sin(theta/2))
b = sqrt(1/2)*(cos(theta/2)-sin(theta/2))
psi = np.array([[a],[0],[0],[0]])+np.array([[0],[0],[0],[b]])

# Calculate Fidelity
psi_T = np.transpose(psi)
A = density_matrix.dot(psi)
Fidelity = psi_T.dot(A)
print("Fidelity:",Fidelity)

NoiseModel:
  Basis gates: ['cx', 'id', 'rz', 'sx']
  Instructions with noise: ['cx']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7]
  Specific qubit errors: [('cx', (0, 4)), ('cx', (1, 5)), ('cx', (2, 6)), ('cx', (3, 7))]
[[ 0.5985+0.j      0.002 +0.0095j -0.0075+0.001j   0.001 +0.j    ]
 [ 0.002 -0.0095j  0.1215+0.j      0.    +0.021j   0.0145-0.003j ]
 [-0.0075-0.001j   0.    -0.021j   0.1045+0.j      0.001 +0.0015j]
 [ 0.001 +0.j      0.0145+0.003j   0.001 -0.0015j  0.1755+0.j    ]]
[[0.5985+0.j]]
